In [1]:
import joblib
import torch
import numpy as np
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from model import DF
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
setup_seed(42)

In [2]:
def create_test_set_AWF_open(features_model,dataset,shot,size,class_num):
    n_query = 70
    clf = torch.nn.Sequential(torch.nn.Linear(2560,class_num)).cuda()
    clf.load_state_dict(torch.load(f'./pre_trained_model/open_model_{dataset}_{shot}.pt'))
    train_name = f'/root/datasets/FSCIL/{dataset}.npz'
    train_dataset = np.load(train_name,allow_pickle=True)
    train_data = train_dataset['data']
    train_labels = train_dataset['labels']
    if( shot == 20 and dataset == 'KNN'):
        X_test = train_data
        y_test = train_labels
    else:
        X_train,X_test, y_train, y_test =train_test_split(train_data,train_labels,test_size=(shot+n_query)*class_num, random_state=42,shuffle=True,stratify=train_labels)
    y_test = np.array([str(lab) for lab in y_test])
    unique = np.unique(y_test)
    labels = []
    proba = []
    for cla in unique:
        inds_train = np.argwhere(y_test==cla)

        samples_train = inds_train.reshape(-1)
        
        query = np.array(X_test[samples_train][shot:])
        query = query.reshape(query.shape[0],1,query.shape[1])

        query = torch.tensor(query,dtype=torch.float32).cuda()
        features_model.eval()
        clf.eval()
        with torch.no_grad():
            proba.extend(softmax(clf(features_model(query)).cpu().numpy()))
            labels.extend([1 for i in range(n_query)])
    print(len(labels))
    train_name = f'/root/datasets/FSCIL/AWF_{size}.npz'
    train_dataset = np.load(train_name,allow_pickle=True)
    train_data = train_dataset['data']
    for i in range(0,size,200):
        query = np.array(train_data[i:i+200])
        query = query.reshape(query.shape[0],1,query.shape[1])
        query = torch.tensor(query,dtype=torch.float32).cuda()
        features_model.eval()
        clf.eval()
        with torch.no_grad():
            proba.extend(softmax(clf(features_model(query)).cpu().numpy()))
            labels.extend([0 for i in range(200)])
    print(len(labels))
    return proba, labels

def cross_entropy(predictions, targets):
    epsilon = 1e-12  # 添加一个小的常数，避免log(0)的情况

    predictions = np.clip(predictions, epsilon, 1.0 - epsilon)  # 限制预测值的范围
    ce = -np.sum(targets * np.log(predictions), axis=1)  # 计算交叉熵

    return ce
def softmax(x):
    e_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # 减去最大值，以避免溢出
    return e_x / np.sum(e_x, axis=1, keepdims=True)

In [12]:
for shot in [1,5,10,15,20]: 
    dataset = 'DF19'
    class_num = 100
    model_path = f'./pre_trained_model/finish_model_{dataset}_{shot}.pt'
    checkpoint = torch.load(model_path)
    feature_model = DF()
    feature_model.cuda()
    feature_model.load_state_dict(checkpoint)
    size = 9000
    
    prob,lab = create_test_set_AWF_open(feature_model,dataset,shot,size,class_num)
    if(size == 400000):
        for i in range(11):
            lab.pop()
        tol = 70*class_num+size-11
    else:
        tol = 70*class_num+size
    prob = np.array(prob)
    ll = np.ones([tol, class_num])
    va = cross_entropy(prob,ll)
    import matplotlib.pyplot as plt
    from sklearn.metrics import precision_recall_curve,roc_auc_score
    precision, recall, thresholds = precision_recall_curve(lab,va)
    # plt.plot(recall, precision)
    # plt.xlabel('Recall')
    # plt.ylabel('Precision')
    # plt.title('Precision-Recall Curve')
    # plt.show()
    print(roc_auc_score(lab,va))
    index = (precision + recall).argmax()
    print('thresh'+str(thresholds[index]))
    print('precision'+str(precision[index]))
    print('recall'+str(recall[index]))

/root/FSCIL/CDWF/model.py:313: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


7000
16000
0.866037507936508
thresh553.2605994939804
precision0.7655434935852249
recall0.7757142857142857


/root/FSCIL/CDWF/model.py:313: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


7000
16000
0.9688481269841269
thresh764.278169721365
precision0.9176140558098331
recall0.8878571428571429


/root/FSCIL/CDWF/model.py:313: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


7000
16000
0.9832069047619048
thresh889.7975144982338
precision0.9551656920077972
recall0.91


/root/FSCIL/CDWF/model.py:313: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


7000
16000
0.9850026190476191
thresh952.524019241333
precision0.9522129098661961
recall0.9251428571428572


/root/FSCIL/CDWF/model.py:313: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


7000
16000
0.9872244126984127
thresh1028.0672616511583
precision0.9642114642114642
recall0.9275714285714286


In [15]:
index = (precision + recall).argmax()
print(thresholds[index])
print(precision[index])
print(recall[index])

1028.0672616511583
0.9642114642114642
0.9275714285714286


In [3]:

for size in [400000]: 
    dataset = 'tor_100w_2500tr'
    shot = 10
    class_num = 100
    model_path = f'./pre_trained_model/finish_model_{dataset}_{10}.pt'
    checkpoint = torch.load(model_path)
    feature_model = DF()
    feature_model.cuda()
    feature_model.load_state_dict(checkpoint)

    prob,lab = create_test_set_AWF_open(feature_model,dataset,shot,size,class_num)
    if(size == 400000):
        for i in range(11):
            lab.pop()
        tol = 70*class_num+size-11
    else:
        tol = 70*class_num+size
    prob = np.array(prob)
    ll = np.ones([tol, class_num])
    va = cross_entropy(prob,ll)
    import matplotlib.pyplot as plt
    from sklearn.metrics import precision_recall_curve,roc_auc_score
    precision, recall, thresholds = precision_recall_curve(lab,va)
    # plt.plot(recall, precision)
    # plt.xlabel('Recall')
    # plt.ylabel('Precision')
    # plt.title('Precision-Recall Curve')
    # plt.show()
    print(roc_auc_score(lab,va))


/root/FSCIL/CDWF/model.py:313: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(m.weight)


7000
407000
0.9583934186761565


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.20246156, 0.19466791,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])